In [3]:
import torch
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from make_dataset import DataGen
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import torch.nn as nn
from patient_split_by_id import *
import numpy as np

/Users/pavankumardharmoju/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/pavankumardharmoju/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <F263C8ED-23C1-35DD-BD33-2CD667C0ED1D> /Users/pavankumardharmoju/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Users/pavankumardharmoju/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
## TODO: get list of image paths and labels
img_labels = pd.read_csv('data/per_scan_data.csv')

In [8]:
img_labels.head()

,scan_name,scan_number,folder_name,patient_id,eye_side,status
0,064_OS_6_GA_6x6_SSOCT_1_168.jpg,168,064_OS_6_GA_6x6_SSOCT_1,64,left,False
1,064_OS_6_GA_6x6_SSOCT_1_202.jpg,202,064_OS_6_GA_6x6_SSOCT_1,64,left,True
2,064_OS_6_GA_6x6_SSOCT_1_397.jpg,397,064_OS_6_GA_6x6_SSOCT_1,64,left,False
3,064_OS_6_GA_6x6_SSOCT_1_204.jpg,204,064_OS_6_GA_6x6_SSOCT_1,64,left,True
4,064_OS_6_GA_6x6_SSOCT_1_44.jpg,44,064_OS_6_GA_6x6_SSOCT_1,64,left,False


In [13]:
#for MAC
all_slices_path = r'/Volumes/fsmresfiles/Ophthalmology/Mirza_Images/AMD/dAMD_GA/all_slices_3'
all_slices = os.listdir(all_slices_path)

In [ ]:
# For windows
# all_slices_path = r'\\fsmresfiles.fsm.northwestern.edu\fsmresfiles\Ophthalmology\Mirza_Images\AMD\dAMD_GA\all_slices_3'
# all_slices = os.listdir(all_slices_path)

In [15]:
all_slices[0:5]

['064_OD_1_GA_6x6_SSOCT_2_365.jpg',
 '064_OD_7_GA_6x6_SSOCT_2_170.jpg',
 '064_OD_3_GA_6x6_SSOCT_1_453.jpg',
 '064_OD_8_GA_6x6_SSOCT_1_256.jpg',
 '341_OS_1_GA_6x6_SSOCT_1_459.jpg']

In [16]:
all_imgs = [item for item in all_slices if item.endswith('.jpg')]

In [17]:
def find_label_patient_id(img_name, img_labels):
   
    name_column = 'scan_name'

    found_row = img_labels[img_labels[name_column] == img_name]
    return found_row['status'].item()

labels = []
for i in range(len(all_imgs)):
    img_name = all_imgs[i]

    label = find_label_patient_id(img_name, img_labels)
    if label == True:
        labels.append(1)
    else:
        labels.append(0)

In [18]:
#dataset
all_imgs_df = pd.DataFrame({'scan_name': all_imgs, 'label': labels})

In [24]:
all_imgs_df.head()

,scan_name,label
0,064_OD_1_GA_6x6_SSOCT_2_365.jpg,0
1,064_OD_7_GA_6x6_SSOCT_2_170.jpg,1
2,064_OD_3_GA_6x6_SSOCT_1_453.jpg,0
3,064_OD_8_GA_6x6_SSOCT_1_256.jpg,1
4,341_OS_1_GA_6x6_SSOCT_1_459.jpg,0


In [19]:
split_settings = 1
# setting 0 = random split
# setting 1 = by patient ID

In [20]:
train_patient_id = ['64', '47', '341']
val_patient_id = ['578', '326']
test_patient_id = ['345', '321', '190']

In [21]:
if split_settings == 1:
    # split by patient ID
    X_train, X_test, y_train, y_test, X_val, y_val = split_by_id(img_labels, all_imgs, labels, train_patient_id,test_patient_id,val_patient_id)
else:
    # split scans random
    X_trainval, X_test, y_trainval, y_test = train_test_split(all_imgs, labels, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42)

In [28]:
X_train.shape

(0,)

In [22]:
train_dataset = DataGen(X_train, y_train, all_slices_path, image_height=1536, image_width=500)
val_dataset = DataGen(X_val, y_val, all_slices_path, image_height=1536, image_width=500)
test_dataset = DataGen(X_test, y_test, all_slices_path, image_height=1536, image_width=500)

In [26]:
print(f"Training Dataset Length: {len(train_dataset)}")
print(f"Validation Dataset Length: {len(val_dataset)}")
print(f"Testing Dataset Length: {len(test_dataset)}")

Training Dataset Length: 0
Validation Dataset Length: 0
Testing Dataset Length: 0


In [23]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

ValueError: num_samples should be a positive integer value, but got num_samples=0